In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import pickle

In [2]:
N_DAYS = 100
N_FAMILIES = 5000
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125
N_OCCUPANCY = 176

data = pd.read_csv('santa-workshop-tour-2019/family_data.csv', index_col='family_id')

FAMILY_SIZE = data.n_people.values

In [3]:
CHOICES = data.drop('n_people', axis=1).values -1
# with open ('santa-workshop-tour-2019/Choice_Table.pkl', 'rb') as fp: CHOICES = pickle.load(fp)

In [4]:
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9 * n
    elif choice == 3:
        penalty = 100 + 9 * n
    elif choice == 4:
        penalty = 200 + 9 * n
    elif choice == 5:
        penalty = 200 + 18 * n
    elif choice == 6:
        penalty = 300 + 18 * n
    elif choice == 7:
        penalty = 300 + 36 * n
    elif choice == 8:
        penalty = 400 + 36 * n
    elif choice == 9:
        penalty = 500 + 36 * n + 199 * n
    else:
        penalty = 500 + 36 * n + 398 * n
    return penalty


def GetPreferenceCostMatrix(data):
    cost_matrix = np.zeros((N_FAMILIES, N_DAYS), dtype=np.float32)
    for i in range(N_FAMILIES):
        desired = data.values[i, :-1]
        cost_matrix[i, :] = get_penalty(FAMILY_SIZE[i], 10)
        for j, day in enumerate(desired):
            cost_matrix[i, day-1] = get_penalty(FAMILY_SIZE[i], j)
    return cost_matrix


def GetAccountingCostMatrix():
    ac = np.zeros((N_OCCUPANCY, N_OCCUPANCY), dtype=np.float32)
    for i,n in enumerate(range(125,301)):
        for j,n_p1 in enumerate(range(125,301)):
            diff = abs(n - n_p1)
            ac[i, j] = max(0, (n - 125) / 400 * n**(0.5 + diff / 50.0))
    return ac

In [5]:
PREFCOST = GetPreferenceCostMatrix(data)
ACCOST = GetAccountingCostMatrix()

In [6]:
PREFCOST = PREFCOST.tolist()
ACCOST = ACCOST.tolist()

In [7]:
# Create a new model
model = gp.Model("santa")

Using license file /home/igor/gurobi.lic
Academic license - for non-commercial use only


In [8]:
disabled_bad_choice = [[1 if d in row[:5] else 0 for d in range(100) ] for r, row in enumerate(CHOICES)]
x_lb  = [[0 for d in range(100) ] for r, row in enumerate(CHOICES)]

In [8]:
%%time
#Preference Cost VARS
x = model.addVars(N_FAMILIES, N_DAYS, ub=1.0, lb=0.0, vtype=GRB.BINARY, name='x')
# Accounting Cost VARS
y = model.addVars(N_DAYS,N_OCCUPANCY,N_OCCUPANCY, ub=1.0, vtype=GRB.BINARY, name='y')

CPU times: user 16.1 s, sys: 613 ms, total: 16.7 s
Wall time: 16.6 s


In [9]:
### Init Solution ###

init = pd.read_csv('solution_68927.csv')
init_day = init[['family_id', 'assigned_day']].values

for f in range(N_FAMILIES):
    for d in range(N_DAYS):
        x[f, d].start =  False
    day = init_day[f, 1]
    x[f, day-1].start = True
model.update()
print(x[0,51].start)   

1.0


In [10]:
%%time
#Preference cost
preference_cost = [PREFCOST[f][d]*x[f,d] for f in range(N_FAMILIES) for d in range(N_DAYS)]
preference_cost = sum(preference_cost)

CPU times: user 2.33 s, sys: 43.8 ms, total: 2.37 s
Wall time: 2.37 s


In [11]:
%%time
#Accounting Cost
acc_cost = [ACCOST[u][v]*y[d,u,v] for u in range(N_OCCUPANCY) for v in range(N_OCCUPANCY) for d in range(N_DAYS)]
acc_cost = sum(acc_cost)

CPU times: user 12.6 s, sys: 176 ms, total: 12.7 s
Wall time: 12.7 s


In [12]:
%%time
model.setObjective(preference_cost + acc_cost, GRB.MINIMIZE)

CPU times: user 10min 46s, sys: 3.19 s, total: 10min 49s
Wall time: 10min 45s


In [14]:
# _arr = []
# with open('Santa_Workshop_Tour_2019/occ_constr_2.txt') as stream:
#     for i, line in enumerate(stream.readlines()):
#         if ',' not in line: continue
#         values = int(line.split(',')[0])
#         _arr.append(values)
# min_arr = _arr[::2]
# max_arr = _arr[1::2]

In [13]:
occupancy={}
for d in range(N_DAYS):
    occupancy[d] = sum([x[f,d]*FAMILY_SIZE[f] for f in range(N_FAMILIES)])
occupancy[d+1] = occupancy[d]

In [14]:
%%time
for f in range(N_FAMILIES):
    model.addConstr(sum([x[f,d] for d in range(N_DAYS)]) == 1)

CPU times: user 1.79 s, sys: 0 ns, total: 1.79 s
Wall time: 1.79 s


In [15]:
%%time
for d in range(N_DAYS):
    model.addConstr(occupancy[d] >= 125)
    model.addConstr(occupancy[d] <= 300)

CPU times: user 972 ms, sys: 0 ns, total: 972 ms
Wall time: 971 ms


In [16]:
count= list(range(125,301))

In [17]:
%%time
for d in range(N_DAYS):
    y_sum_u = sum( y[d,u,v]*count[u] for u in range(N_OCCUPANCY) for v in range(N_OCCUPANCY) )
    y_sum_v = sum( y[d,u,v]*count[v] for u in range(N_OCCUPANCY) for v in range(N_OCCUPANCY) )
    model.addConstr(y_sum_u == occupancy[d])
    model.addConstr(y_sum_v == occupancy[d+1])
    
    y_sum = sum( y[d,u,v] for v in range(N_OCCUPANCY) for u in range(N_OCCUPANCY) )
    model.addConstr(y_sum == 1)

CPU times: user 49.1 s, sys: 27.9 ms, total: 49.1 s
Wall time: 49.1 s


In [18]:
%%time
for d in range(N_DAYS -1):
    for t in range(N_OCCUPANCY):
        y_sum_u = sum( y[d,u,t] for u in range(N_OCCUPANCY) )
        y_sum_v = sum( y[d+1,t,v] for v in range(N_OCCUPANCY) )
        model.addConstr(y_sum_u == y_sum_v)
        

CPU times: user 28.6 s, sys: 116 ms, total: 28.7 s
Wall time: 28.8 s


In [19]:
## Callbacks ##
def simple_mst_writer(model, mstfilename, nodecnt, obj):
    mstfile = open(mstfilename, 'w')
    varlist = model.getVars()
    soln = model.cbGetSolution(varlist) #  cbGetSolution
    mstfile.write('# MIP start from soln at node %d obj %e\n' % (nodecnt, obj))
    for var, soln in zip(varlist, soln):
        mstfile.write('%s %.3e\n' % (var.VarName, soln))
    mstfile.close()


def mycallback(model, where):
    if where == GRB.callback.MIPSOL: # MIPSOL
        obj = model.cbGet(GRB.callback.MIPSOL_OBJ)
        nodecnt = int(model.cbGet(GRB.callback.MIPSOL_NODCNT))
        print('Found solution at node', nodecnt, 'obj', obj)
        simple_mst_writer(model, 'sol.mst', nodecnt, obj)

In [20]:
## Model Param
model.setParam('MIPGap', 0) 
model.setParam('Seed', 133) 

Changed value of parameter MIPGap to 0.0
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter Seed to 133
   Prev: 0  Min: 0  Max: 2000000000  Default: 0


In [21]:
try:
    model.optimize(mycallback)

    print('Obj: %g' % model.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 22924 rows, 3597600 columns and 17926048 nonzeros
Model fingerprint: 0x868b1126
Variable types: 0 continuous, 3597600 integer (3597600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-02, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Found solution at node 0 obj 68927.3555410821
User MIP start produced solution with objective 68927.4 (20.66s)
Found solution at node 0 obj 68927.3555410821
Loaded user MIP start with objective 68927.4
Processed MIP start in 35.02 seconds

Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 0 columns (presolve time = 10s) ...
Presolve removed 0 rows and 0 columns (presolve time = 15s) ...
Presolve removed 100 rows and 0 columns (presolve time = 20s) ...
Presolve rem

In [23]:
print('Obj: %g' % model.objVal)

AttributeError: Unable to retrieve attribute 'objVal'

In [23]:
## Generate new submit file with best solution ##
days = []
f = 0
for v in model.getVars()[:500000]:
    f += 1
    x = v.x
    if int(x) == 1:
        days.append(f)
    if f == 100:
        f = 0

init['assigned_day'] =  days
init.to_csv('best_solution.csv', index=False)

In [16]:
days = []
fa = 0
init = pd.read_csv('santa-workshop-tour-2019/solution_68939.csv')
with open('Santa_Workshop_Tour_2019/sol.mst') as f:
    lines = f.readlines()[1:500001]
    for line in lines:
        fa += 1
        x = line.split(' ')[1]
        if '1.00' in x:
            days.append(fa)
        if fa == 100:
            fa = 0
            
init['assigned_day'] =  days
init.to_csv('solution_68888.csv', index=False)

In [15]:
len(days)

0

TypeError: bad operand type for abs(): 'gurobipy.LinExpr'